# Launch ComfyUI with TensorRT on Brev!

Welcome!

In this notebook, we'll walk through how to use ComfyUI with TensorRT. ComfyUI is a powerful GUI for Stable Diffusion models. It uses a node-based architecture that allows users to construct very complex image/video generation workflows by connecting different nodes, each representing a different function or operation!

TensorRT engines contain state-of-the-art optimizations to performance efficiently on NVIDIA GPUs. Once the TensorRT engine is built, you can implement it in your ComfyUI workflow to take advantage of the ultra-fast performance on your RTX GPU!

In [ ]:
# Environment Setup

from pathlib import Path

OPTIONS = {}

WORKSPACE = 'ComfyUI'
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

In [ ]:
%%bash
if ! [ -d WORKSPACE ]; then
    echo "Starting initial setup of ComfyUI..."
    git clone https://github.com/comfyanonymous/ComfyUI
    if [ -d ComfyUI ]; then
        echo "Repository cloned successfully into 'ComfyUI'."
        cd ComfyUI
    else
        echo "Failed to clone repository."
    fi
else
    echo "Workspace directory already exists."
fi

In [ ]:
if OPTIONS['UPDATE_COMFY_UI']:
    !echo "Updating ComfyUI..."
    !git pull

!echo "Installing dependencies..."
!pip install accelerate
!pip install einops 
!pip install transformers
!pip install safetensors>=0.3.0 
!pip install aiohttp 
!pip install pyyaml 
!pip install Pillow 
!pip install scipy 
!pip install tqdm 
!pip install psutil
!pip install torch==2.2.2 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install torchsde
!pip install kornia>=0.7.1 spandrel
!pip install typer
!pip install diffusers

### Optional, run this if you want to download ComfyUI manager

In [ ]:

import os

if not os.path.isfile("ComfyUI-Manager/check.sh"):
    !chmod 755 ComfyUI-Manager/check.sh
if not os.path.isfile("ComfyUI-Manager/scan.sh"):
    !chmod 755 ComfyUI-Manager/scan.sh
if not os.path.isfile("ComfyUI-Manager/node_db/dev/scan.sh"):
    !chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
if not os.path.isfile("ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh"):
    !chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
if not os.path.isfile("ComfyUI-Manager/scripts/install-comfyui-venv-win.bat"):
    !chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

if not os.path.isdir("ComfyUI-Manager"):
    print("-= Initial setup ComfyUI-Manager =-")
    !git clone https://github.com/ltdrdata/ComfyUI-Manager 

%cd ComfyUI-Manager
!git pull

%cd ..

%cd ~/verb-workspace

In [ ]:
import shutil
import os

# Move the ComfyUI-Manager folder to the custom_nodes directory
source_manager_folder = '/root/verb-workspace/ComfyUI-Manager'
# source_manager_folder = '/home/ubuntu/verb-workspace/ComfyUI-Manager' (if you run into error, try this)
destination_manager_folder = '/root/verb-workspace/ComfyUI/custom_nodes/ComfyUI-Manager'
# destination_manager_folder = '/home/ubuntu/verb-workspace/ComfyUI/custom_nodes/ComfyUI-Manager' (if you run into error, try this)

if os.path.exists(destination_manager_folder):
    shutil.rmtree(destination_manager_folder)

shutil.move(source_manager_folder, destination_manager_folder)

assert os.path.exists(destination_manager_folder), "Folder move failed for ComfyUI-Manager"
print(f'Folder ComfyUI-Manager moved to {destination_manager_folder}')

In [ ]:
if 'INSTALL_CUSTOM_NODES_DEPENDENCIES' in OPTIONS and OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
    print("-= Install custom nodes dependencies =-")
    !pip install GitPython
    !python /root/verb-workspace/ComfyUI/custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies

### Time to download some models (safetensors + checkpoints)

In [ ]:
!pip install huggingface_hub

In [ ]:
!wget -c https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -P ./ComfyUI/models/checkpoints/

In [ ]:
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./ComfyUI/models/vae/

#### We're going to build the TensorRT engine for the model below (SDXL Turbo)

In [ ]:
from diffusers import StableDiffusionPipeline

# Download the model
!wget -O ./ComfyUI/models/checkpoints/sd_xl_turbo_1.0_fp16.safetensors https://huggingface.co/stabilityai/sdxl-turbo/resolve/main/sd_xl_turbo_1.0_fp16.safetensors

# Load the model from the local path
pipeline = StableDiffusionPipeline.from_single_file(
    "./ComfyUI/models/checkpoints/sd_xl_turbo_1.0_fp16.safetensors"
)

In [ ]:
!cd ./ComfyUI/custom_nodes && git clone https://github.com/comfyanonymous/ComfyUI_TensorRT

In [ ]:
!cd ./ComfyUI/custom_nodes/ComfyUI_TensorRT && pip install -r requirements.txt

#### This next cell will pull out 2 .JSON files you'll need to:

1) Build the TensorRT Engine for SDXL Turbo Model
2) Use the engine in the ComfyUI Workflow

In [ ]:
source_folder_path = '/root/verb-workspace/ComfyUI/custom_nodes/ComfyUI_TensorRT/workflows'
# source_folder_path = '/home/ubuntu/verb-workspace/ComfyUI/custom_nodes/ComfyUI_TensorRT/workflows' (if you run into error, try this)
destination_folder_path = '/root/verb-workspace/'
# destination_folder_path = '/home/ubuntu/verb-workspace' (if you run into error, try this)

json_files = [
    'Create_SDXL_Turbo_TRT_Static.json',
    'Build.TRT.Engine_SDXL_Turbo_Static.json'
]

# Move the workflows folder to the main directory
destination_workflows_path = os.path.join(destination_folder_path, 'workflows')
if os.path.exists(destination_workflows_path):
    shutil.rmtree(destination_workflows_path)

shutil.move(source_folder_path, destination_folder_path)

# Move the JSON files out of the workflows folder into the main directory
for json_file in json_files:
    source_json_path = os.path.join(destination_workflows_path, json_file)
    destination_json_path = os.path.join(destination_folder_path, json_file)
    shutil.move(source_json_path, destination_json_path)
    
    assert os.path.exists(destination_json_path), f"File move failed for {json_file}"
    print(f'File {json_file} moved to {destination_json_path}')

In [ ]:
import subprocess
import threading
import time
import socket

def check_server(port):
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                print("\nServer is up and running at http://129.80.69.99:{}".format(port))
                break
            time.sleep(0.5)

threading.Thread(target=check_server, daemon=True, args=(8188,)).start()

In [ ]:
!python /root/verb-workspace/ComfyUI/main.py --listen 0.0.0.0 --port 8188 --dont-print-server --disable-xformers
# !python /home/ubuntu/verb-workspace/ComfyUI/main.py --listen 0.0.0.0 --port 8188 --dont-print-server --disable-xformers

### We just started the ComfyUI server! 

##### We can access it by going to the Brev instance page and clicking on the Port 8188 URL

![image.png](https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/notebooks/Screenshot_2024-06-04_at_13.00.59.png?t=2024-06-04T20%3A01%3A47.076Z)

#### Download the 2 JSON files we pulled into the main directory to your local computer 
##### Files you need to download: 'Create_SDXL_Turbo_TRT_Static.json', 'Build.TRT.Engine_SDXL_Turbo_Static.json'

##### The ComfyUI server should be running, so to begin, click on the URL at Port 8188 (as shown above) 

##### Once you see the default workflow, we can use the `Load` Button to bring in the 'Build.TRT.Engine_SDXL_Turbo_Static.json' file from your laptop. 

![image.png](https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/notebooks/LoadBuildjson.png?t=2024-06-04T18%3A01%3A20.036Z)

##### Make sure to upload the 'Build.TRT.Engine_SDXL_Turbo_Static.json' file from your laptop now!

##### Click `Queue Prompt` to begin the workflow. This will build the TensorRT engine (takes a couple minutes)

![image.png](https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/notebooks/BuildTRTEngine.png)

##### Now that the engine is built, it's time to actually run a ComfyUI workflow with it and see how fast it is yourself. Click the `Load` Button again to bring in the 'Create_SDXL_Turbo_TRT_Static.json' file. 

![image.png](https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/notebooks/loadCreatejson.png?t=2024-06-04T18%3A13%3A54.983Z)


##### Now, click `Queue Prompt` to run this workflow and generate images with SDXL Turbo!

![image.png](https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/notebooks/TRTComfyUIWorkflow.png)

##### Here was my output using the TRT Engine in my ComfyUI workflow!

![image.png](https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/notebooks/Screenshot_2024-06-04_at_11.15.57.png?t=2024-06-04T18%3A16%3A50.463Z)

In [ ]:
def check_server(port):
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                print("\nServer is up and running at http://129.80.69.99:{}".format(port))
                break
            time.sleep(0.5)

threading.Thread(target=check_server, daemon=True, args=(8188,)).start()